In [1]:
import sqlite3
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

from IPython.display import display

In [2]:
db = sqlite3.connect('L20DB_demo.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys = 1")

# Create candidates table

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit()

# Create contributors table

cursor.execute('''CREATE TABLE contributors (
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    last_name TEXT,
    first_name TEXT, 
    middle_name TEXT,
    street_1 TEXT,
    street_2 TEXT,
    city TEXT,
    state TEXT,
    zip TEXT,
    amount REAL, 
    date DATETIME,
    candidate_id INTEGER NOT NULL,
    FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

In [3]:
# Populate candidates table

with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)

In [4]:
# Populate contributors table

with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

In [5]:
# Visualize the tables

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [row[i] for row in q]
    return pd.DataFrame.from_dict(framelist)

In [6]:
# Displaying the candidates table

candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
query = '''SELECT * FROM candidates'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party
0,16,Mike,Huckabee,,R
1,20,Barack,Obama,,D
2,22,Rudolph,Giuliani,,R
3,24,Mike,Gravel,,D
4,26,John,Edwards,,D
5,29,Bill,Richardson,,D
6,30,Duncan,Hunter,,R
7,31,Dennis,Kucinich,,D
8,32,Ron,Paul,,R
9,33,Joseph,Biden,,D


In [7]:
# Displaying the contributors table

contributor_cols =[col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]
query = '''SELECT * FROM contributors'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,1,Agee,Steven,,549 Laurel Branch Road,,Floyd,VA,24091,500.0,2007-06-30,16
1,2,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,250.0,2007-05-16,16
2,3,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,50.0,2007-06-18,16
3,4,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,100.0,2007-06-21,16
4,5,Akin,Charles,,10187 Sugar Creek Road,,Bentonville,AR,72712,100.0,2007-06-16,16
...,...,...,...,...,...,...,...,...,...,...,...,...
170,171,ABESHAUS,MERRILL,M.,1801 N. HEREFORD DRIVE,,FLAGSTAFF,AZ,860011121,120.0,2008-01-16,37
171,172,ABRAHAM,GEORGE,,P.O. BOX 1504,,LAKE CHARLES,LA,706021504,800.0,2008-01-17,37
172,173,ABRAHAMSON,PETER,J.,1030 W. ROSCOE STREET,,CHICAGO,IL,606572207,50.0,2008-01-25,37
173,174,ABRAHAM,SALEM,A.,P.O. BOX 7,,CANADIAN,TX,790140007,1000.0,2008-01-17,37


In [8]:
query = '''SELECT * FROM candidates WHERE middle_init <>""'''
viz_tables(candidate_cols, query)
print("{} candidates have a middle initial.".format(viz_tables(candidate_cols, query).shape[0]))

3 candidates have a middle initial.


In [9]:
# Display contributors where state is PA
query = '''SELECT * FROM contributors WHERE state = "PA" '''
viz_tables(contributor_cols, query)

# Display the contributors where amount contributed is greater than $1000
query = '''SELECT * FROM contributors WHERE amount > 1000'''
viz_tables(contributor_cols, query)

# Display the contributors from UT where amount contributed is greater than $1000
query = '''SELECT * FROM contributors WHERE state = "UT" AND amount > 1000'''
viz_tables(contributor_cols, query)

# Display the contributors who did not list their state
query = '''SELECT * FROM contributors WHERE state = ""'''
viz_tables(contributor_cols, query)

# Display the contributors from "WA" or "PA"
query = '''SELECT * FROM contributors WHERE state IN ("WA", "PA")'''
viz_tables(contributor_cols, query)

# Display the contributors between $100 and $200
query = '''SELECT * FROM contributors where amount > 100 AND amount < 200'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,134,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,,PEARLAND,TX,775816547,150.0,2007-08-09,35
1,162,ABACHERLI,SHIRLEY,M.,29875 NEWPORT ROAD,,MENIFEE,CA,925849524,150.0,2008-01-28,37
2,171,ABESHAUS,MERRILL,M.,1801 N. HEREFORD DRIVE,,FLAGSTAFF,AZ,860011121,120.0,2008-01-16,37


In [10]:
# Sort descending by ID
query = '''SELECT * FROM candidates ORDER BY id DESC'''
viz_tables(candidate_cols, query)

# Sort ascending by last_name
query = '''SELECT * FROM candidates ORDER BY last_name'''
viz_tables(candidate_cols, query)

# Sort descending where contribution btwn 1000 & 5000
query = '''SELECT * FROM contributors WHERE amount > 1000 AND amount < 5000 ORDER BY amount DESC'''
viz_tables(contributor_cols, query)

# Sort contributors between 1000 & 5000 with candidate_id and amount in descending order
query = ''' SELECT * FROM contributors WHERE amount > 1000 AND amount < 5000 ORDER BY candidate_id ASC, amount DESC'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16
1,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16
2,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16
3,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16
4,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16
5,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20
6,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20
7,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.0,2007-09-30,20
8,46,Buchanan,John,,2025 NW 29th Rd,,Boca Raton,FL,334316303,1300.0,2007-08-09,20
9,136,ABRAMOWITZ,NIRA,,411 HARBOR ROAD,,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35


In [11]:
# Selecting specific columns
query = '''SELECT last_name, party FROM candidates'''
viz_tables(['last_name', 'party'], query)

# Using distinct
query = '''SELECT DISTINCT party FROM candidates'''
viz_tables(['party'], query)

# First and last name of contributors, distinct
query = '''SELECT DISTINCT first_name, last_name FROM contributors'''
viz_tables(['first_name', 'last_name'], query)

,first_name,last_name
0,Steven,Agee
1,Don,Ahrens
2,Charles,Akin
3,Mike,Akin
4,Rebecca,Akin
...,...,...
121,DENIS,ABERCROMBIE
122,MERRILL,ABESHAUS
123,GEORGE,ABRAHAM
124,PETER,ABRAHAMSON


In [12]:
# Altering Candidates Table

cursor.execute('''ALTER TABLE candidates ADD COLUMN full_name TEXT''')
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
viz_tables(candidate_cols, '''SELECT * FROM candidates''')

# Populate full names
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
query = '''SELECT id, last_name, first_name FROM candidates'''
full_name_and_id = [(attr[1] + ", " + attr[2], attr[0]) for attr in cursor.execute(query).fetchall()]

update = '''UPDATE candidates SET full_name = ? WHERE id = ?'''
for rows in full_name_and_id:
    cursor.execute(update, rows)

query = '''SELECT * FROM candidates'''
viz_tables(candidate_cols, query)

# Another update

update = '''UPDATE candidates SET full_name = "WINNER" WHERE last_name = "Obama"'''
cursor.execute(update)
update = '''UPDATE candidates SET full_name = "RUNNER-UP" WHERE last_name = "McCain"'''
cursor.execute(update)
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party,full_name
0,16,Mike,Huckabee,,R,"Huckabee, Mike"
1,20,Barack,Obama,,D,WINNER
2,22,Rudolph,Giuliani,,R,"Giuliani, Rudolph"
3,24,Mike,Gravel,,D,"Gravel, Mike"
4,26,John,Edwards,,D,"Edwards, John"
5,29,Bill,Richardson,,D,"Richardson, Bill"
6,30,Duncan,Hunter,,R,"Hunter, Duncan"
7,31,Dennis,Kucinich,,D,"Kucinich, Dennis"
8,32,Ron,Paul,,R,"Paul, Ron"
9,33,Joseph,Biden,,D,"Biden, Joseph"


In [13]:
# Altering Contributors Table

cursor.execute('''ALTER TABLE contributors ADD COLUMN full_name TEXT''')
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]
viz_tables(contributor_cols, '''SELECT * FROM contributors''')

contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")] # regenerate columns with full_name
query = '''SELECT id, last_name, first_name FROM contributors''' # Select a few columns
full_name_and_id = [(attr[1] + ", " + attr[2], attr[0]) for attr in cursor.execute(query).fetchall()] # List of tuples: (full_name, id)

update = '''UPDATE contributors SET full_name = ? WHERE id = ?''' # Update the table
for rows in full_name_and_id:
    cursor.execute(update, rows)

query = '''SELECT * FROM contributors'''
viz_tables(contributor_cols, query)

# Exercise TOO MUCH

update = '''UPDATE contributors SET full_name = "TOO MUCH" WHERE amount > 1000'''
cursor.execute(update)
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,full_name
0,1,Agee,Steven,,549 Laurel Branch Road,,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven"
1,2,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don"
2,3,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don"
3,4,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don"
4,5,Akin,Charles,,10187 Sugar Creek Road,,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,171,ABESHAUS,MERRILL,M.,1801 N. HEREFORD DRIVE,,FLAGSTAFF,AZ,860011121,120.0,2008-01-16,37,"ABESHAUS, MERRILL"
171,172,ABRAHAM,GEORGE,,P.O. BOX 1504,,LAKE CHARLES,LA,706021504,800.0,2008-01-17,37,"ABRAHAM, GEORGE"
172,173,ABRAHAMSON,PETER,J.,1030 W. ROSCOE STREET,,CHICAGO,IL,606572207,50.0,2008-01-25,37,"ABRAHAMSON, PETER"
173,174,ABRAHAM,SALEM,A.,P.O. BOX 7,,CANADIAN,TX,790140007,1000.0,2008-01-17,37,"ABRAHAM, SALEM"


In [16]:
# # Aggregation

contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]
# function = '''SELECT *, MAX(amount) AS max_amount FROM contributors'''
# viz_tables(contributor_cols, function)

# Modified

query = '''SELECT *, MAX(amount) AS max_amount FROM contributors'''
max_amount = [attr for attr in cursor.execute(query).fetchall()]
print(max_amount)
max_amount[0][-1]
# viz_tables(contributor_cols, query)


[(31, 'Buckel', 'Linda', '', 'PO Box 683130', '', 'Park City', 'UT', '840683130', 4600.0, '2007-08-14', 20, 'TOO MUCH', 4600.0)]


4600.0

In [54]:
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]

# Count the total donations over 1000
query = '''SELECT COUNT(*) FROM contributors WHERE amount > 1000'''
count_amount = [attr for attr in cursor.execute(query).fetchall()]
count_amount[0][0]

# Calculate the average positive donation
query = '''SELECT AVG(amount) FROM contributors where amount > 0'''
average_amount = [attr for attr in cursor.execute(query).fetchall()]
average_amount[0][0]

# Average positive donation from each state
query = '''SELECT state, SUM(amount) FROM contributors WHERE amount > 0 GROUP BY state'''
state_average = cursor.execute(query).fetchall()

framelist = dict()

for i, col in enumerate(['State', 'Average Contribution']):
    framelist[col] = [row[i] for row in state_average]

pd.DataFrame.from_dict(framelist)

# Deletion example
deletion = '''DELETE FROM contributors WHERE last_name = "Ahrens"'''

,State,Average Contribution
0,AK,1210.00
1,AR,14200.00
2,AZ,120.00
3,CA,4260.80
4,CT,2300.00
5,DC,450.09
6,FL,10150.00
7,IA,250.00
8,IL,250.00
9,LA,1300.00


In [63]:
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]

# Limit functionality

query = '''SELECT * FROM candidates LIMIT 3'''
viz_tables(candidate_cols, query)

query = '''SELECT * FROM candidates LIMIT 4 OFFSET 5'''
viz_tables(candidate_cols, query)

query = '''SELECT * FROM candidates ORDER BY last_name LIMIT 4 OFFSET 5'''
viz_tables(candidate_cols, query)

# Querying ten most generous donors
query = '''SELECT * FROM contributors ORDER BY amount DESC LIMIT 10'''
viz_tables(contributor_cols, query)

# # Querying ten least generous donors with positive contributions
query = '''SELECT * FROM contributors ORDER BY amount ASC LIMIT 10'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,full_name
0,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20,TOO MUCH
1,160,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37,TOO MUCH
2,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16,TOO MUCH
3,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16,TOO MUCH
4,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16,TOO MUCH
5,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20,TOO MUCH
6,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.0,2007-09-30,20,TOO MUCH
7,136,ABRAMOWITZ,NIRA,,411 HARBOR ROAD,,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35,TOO MUCH
8,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16,TOO MUCH
9,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16,TOO MUCH


In [ ]:
db.commit()
db.close()